In [1]:
#Instalamos la libreria de STREAMLIT
%pip install streamlit

In [2]:
#Instalamos la libreria de PLOTLY
%pip install plotly

In [3]:
#Instalamos librería para análisis de variables categóricas
%pip install funpymodeling

In [4]:

#Instalamos el tunnel local (node.js)
#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versión más recomendada
!npm install localtunnel


up to date, audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [5]:
%pip install streamlit-option-menu

In [13]:
######PRUEB"###########

%%writefile app.py
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from streamlit_option_menu import option_menu
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
import scipy.stats as stats
import funpymodeling as fp


# 1. Agregar la función de CSS para fondo después de las importaciones
def agregar_fondo(imagen_url):
    st.markdown(
        f"""
        <style>
        .stApp {{
            background-image: url("{imagen_url}");
            background-size: cover;
            background-position: center;
            background-attachment: fixed;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

# Configuración de la página con iconos y layout
st.set_page_config(
    page_title="Paciente 08",
    page_icon="🏠",
    layout="wide",
    initial_sidebar_state="expanded"
)


# Cargar los datos
@st.cache_data
def load_data():
    # Cargar los archivos por separado
    data = pd.read_csv('Paciente08.csv', encoding='utf-8')
    return data

@st.cache_data
def load_socio_data():
    socio_data = pd.read_csv('SocioLimpio.csv', encoding='utf-8')
    return socio_data

# Preprocesar datos
def preprocess_data(data):
    data_cuant = data.copy()
    data_cuant = data_cuant.select_dtypes(include=[np.number])
    return data_cuant

# Preprocesar datos de SocioLimpio
def preprocess_socio_data(socio_data):
    socio_categ = data.select_dtypes(include=['object'])  # Seleccionar solo variables categóricas
    return socio_categ

# Funciones de regresión no lineal
def cuadratica(x, a, b, c):
    return a * x**2 + b * x + c

def exponencial(x, a, b, c):
    return a * np.exp(-b * x) + c

def polinomio_inverso(x, a, b, c):
    return a / b * x**2 + c * x

def valor_absoluto(x, a, b, c):
    return abs(a * x**2 + b * x + c)

def inversa(x, a):
    return 1 / (a * x)

def senoidal(x, a, b):
    return a * np.sin(x) + b

def tangencial(x, a, b):
    return a * np.tan(x) + b

def cociente_polinomios(x, a, b, c):
    return (a * x**2 + b) / (c * x)

def logaritmica(x, a, b):
    return a * np.log(x) + b

def producto_coeficientes(x, a, b, c):
    return (a + b + c) * x

def cuadratica_inversa(x, a):
    return 1 / (a * x**2)


# Función de graficar heatmap personalizado con variables seleccionadas
def plot_custom_heatmap(data, selected_vars):
    if len(selected_vars) < 2:
        st.warning("Selecciona al menos dos variables para generar el heatmap.")
        return None
    else:
        corr_matrix = data[selected_vars].corr().abs()  # Aplicar valor absoluto
        fig = px.imshow(
            corr_matrix,
            text_auto=True,
            color_continuous_scale='RdBu_r',
            title="Heatmap de Correlaciones (Valor Absoluto)"
        )

        return fig


# Función para mostrar boxplot
def plot_boxplot(data, y_var):
    fig = px.box(data, y=y_var, title=f"Boxplot de {y_var}", color_discrete_sequence=['#636EFA'])
    return fig

# Función para la regresión no lineal y gráfica
def plot_regression(data, x_var, y_var, func):
    x = data[x_var]
    y = data[y_var]
    try:
        popt, _ = curve_fit(func, x, y)
        x_line = np.linspace(x.min(), x.max(), 100)
        y_line = func(x_line, *popt)

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Datos', marker=dict(color='rgba(0, 150, 255, .8)')))
        fig.add_trace(go.Scatter(x=x_line, y=y_line, mode='lines', name='Regresión', line=dict(color='rgba(255, 0, 0, .8)', width=2)))

        r2 = r2_score(y, func(x, *popt))
        R = np.sqrt(r2)  # Coeficiente de correlación
        return fig, r2, R
    except:
        st.error(f"No se pudo ajustar la regresión para {y_var} vs {x_var}")
        return None, None, None

# Función para la regresión lineal
def plot_linear_regression(data, x_var, y_var):
    x = data[x_var].values.reshape(-1, 1)
    y = data[y_var].values
    try:
        from sklearn.linear_model import LinearRegression
        model = LinearRegression()
        model.fit(x, y)
        y_pred = model.predict(x)

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x.flatten(), y=y, mode='markers', name='Datos', marker=dict(color='rgba(0, 150, 255, .8)')))
        fig.add_trace(go.Scatter(x=x.flatten(), y=y_pred, mode='lines', name='Regresión Lineal', line=dict(color='rgba(255, 0, 0, .8)', width=2)))

        r2 = r2_score(y, y_pred)
        R = np.sqrt(r2)  # Coeficiente de correlación
        return fig, r2, R
    except:
        st.error(f"No se pudo ajustar la regresión para {y_var} vs {x_var}")
        return None, None, None

# Dashboard principal
def main():
    st.markdown("<h1 style='text-align: center; color: white;'>ANÁLISIS PACIENTE 08</h1>", unsafe_allow_html=True)

    # Cargar y preprocesar datos
    data = load_data()
    data_cuant = preprocess_data(data)


    # Sidebar para navegación
    with st.sidebar:
        selected = option_menu("Menú Principal",
                              ["Resumen", "Correlaciones", "Análisis Univariado",
                              "Regresión Lineal", "Regresión Lineal Múltiple",
                              "Regresión No Lineal", "Visualizaciones Adicionales",
                              "Regresión logística"],
                              icons=[
                                  'house', 'link', 'bar-chart',  # Cambiado ícono de "Correlaciones" a 'link'
                                  'line-chart', 'layers',  # Cambiados íconos de regresión lineal y múltiple
                                  'graph-up-arrow', 'palette', 'diagram-2'
                              ],
                              menu_icon="cast", default_index=0)


    st.markdown("<p style='text-align: center;'>E Q U I P O  2 </p>", unsafe_allow_html=True)


    if selected == "Resumen":
        st.subheader("📈 Resumen de Datos Numéricos")

        # Contenedor de dos columnas para el texto y la tabla
        col1, col2 = st.columns([1, 2])  # Proporción: 1/3 texto, 2/3 tabla

        with col1:
            # Mostrar texto creativo en blanco
            st.markdown(
                """
                <div style="color: white; font-family: 'Arial'; font-size: 18px;
                            background-color: rgba(0, 0, 0, 0.8); padding: 20px;
                            border-radius: 10px;">
                    <strong>Tecnológico de Monterrey</strong><br><br>
                    Instituto Nacional de Astrofísica, Óptica y Electrónica (INAOE)
                </div>
                """,
                unsafe_allow_html=True
            )

        with col2:
            # Calcular frecuencias de cada emoción
            emociones = ['01_C', '02_A', '03_D', '04_M']
            resumen = pd.DataFrame({
                'Emoción': ['Cansancio', 'Ansiedad', 'Dolor', 'Motivación'],
                'Presente': [data[emo].value_counts().get(1, 0) for emo in emociones],
                'Ausente': [data[emo].value_counts().get(-1, 0) for emo in emociones]
            })

            # Crear tabla HTML sin la barra negra superior y con alineación a la derecha
            st.markdown(
                """
                <style>
                .custom-table {
                    background-color: black;
                    color: white;
                    border-radius: 10px;
                    padding: 10px;
                    width: 80%;  /* Ajustar la anchura */
                    margin-left: auto;  /* Centrar hacia la derecha */
                    margin-right: auto;
                    text-align: left;
                    border-collapse: collapse;
                }
                .custom-table th, .custom-table td {
                    padding: 10px;
                    border-bottom: 1px solid white;
                }
                </style>
                """, unsafe_allow_html=True
            )

            # Generar HTML para la tabla
            table_html = resumen.to_html(classes='custom-table', index=False)

            # Mostrar la tabla personalizada
            st.markdown(table_html, unsafe_allow_html=True)

        # Gráficos juntos: Distribución y Comparación de emociones
        col3, col4 = st.columns(2)

        with col3:
            st.subheader("📊 Distribución de Emoción Seleccionada")

            emocion_seleccionada = st.selectbox(
                "Selecciona una emoción para la gráfica de pastel", resumen['Emoción']
            )
            valores = resumen[resumen['Emoción'] == emocion_seleccionada][['Presente', 'Ausente']].values.flatten()

            fig_pie = go.Figure(data=[go.Pie(
                labels=['Presente', 'Ausente'],
                values=valores,
                hole=0.4,
                marker=dict(colors=['#28A745', '#DC3545']),
                hoverinfo='label+percent',
                textinfo='value',
            )])

            fig_pie.update_layout(
                title=dict(
                    text=f"Distribución de {emocion_seleccionada}",
                    font=dict(size=18),
                    x=0.5
                ),
                showlegend=True,
                legend=dict(orientation='h', y=-0.1, x=0.5, xanchor='center'),
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)'
            )

            st.plotly_chart(fig_pie, use_container_width=True)

        with col4:
            st.subheader("📊 Comparación de Emociones Presentes")

            fig_bar = px.bar(
                resumen, x='Emoción', y='Presente',
                title="Comparación de emociones presentes",
                text='Presente',
                color='Emoción',
                color_discrete_sequence=px.colors.qualitative.Plotly,
            )

            fig_bar.update_traces(
                texttemplate='%{text}', textposition='outside',
                marker=dict(line=dict(color='black', width=1))
            )

            fig_bar.update_layout(
                title=dict(
                    text="Comparación de emociones presentes",
                    font=dict(size=18),
                    x=0.5
                ),
                xaxis=dict(title='', tickfont=dict(size=14)),
                yaxis=dict(title='Frecuencia', tickfont=dict(size=14)),
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)'
            )

            st.plotly_chart(fig_bar, use_container_width=True)


    elif selected == "Correlaciones":
        st.subheader("🔗 Mapa de Correlaciones")

        # Excluir columnas específicas del DataFrame
        variables = [col for col in data_cuant.columns if col not in ['TimeWindow', 'User']]
        # Multiselect con las columnas filtradas
        selected_vars = st.multiselect("Selecciona las variables para el Heatmap", variables, default=variables[:5])


        if st.checkbox("Mostrar Heatmap de Correlaciones"):
            fig = plot_custom_heatmap(data_cuant, selected_vars)
            if fig:
                st.plotly_chart(fig, use_container_width=True)

        st.subheader("📊 Tabla de Correlaciones")
        if selected_vars:
            corr_table = data_cuant[selected_vars].corr()
            st.dataframe(corr_table)

    elif selected == "Regresión No Lineal":
        st.subheader("📉 Análisis de Regresión No Lineal")


        x_var = st.selectbox("Seleccione Variable Independiente X", data.columns)
        y_var = st.selectbox("Seleccione Variable Dependiente Y", data.columns)

        formula = st.selectbox("Seleccione el tipo de regresión", ['Cuadrática', 'Exponencial', 'Polinomio Inverso', 'Valor Absoluto', 'Inversa', 'Senoidal', 'Tangencial', 'Cociente de Polinomios', 'Logarítmica', 'Producto de Coeficientes', 'Cuadrática Inversa'])

        funciones_regresion = {
            "Cuadrática": cuadratica,
            "Exponencial": exponencial,
            "Polinomio Inverso": polinomio_inverso,
            "Valor Absoluto": valor_absoluto,
            "Inversa": inversa,
            "Senoidal": senoidal,
            "Tangencial": tangencial,
            "Cociente de Polinomios": cociente_polinomios,
            "Logarítmica": logaritmica,
            "Producto de Coeficientes": producto_coeficientes,
            "Cuadrática Inversa": cuadratica_inversa
        }

        if st.button("Generar Regresión"):
            fig, r2, R = plot_regression(data_cuant, x_var, y_var, funciones_regresion[formula])
            if fig:
                st.plotly_chart(fig, use_container_width=True)
                st.metric("Coeficiente de Determinación (R²)", f"{r2:.4f}")
                st.metric("Coeficiente de Correlación (R)", f"{R:.4f}")

    elif selected == "Regresión Lineal":
        st.subheader("📈 Análisis de Regresión Lineal")

        y_var = st.selectbox("Seleccione Variable Dependiente (Y)", ["01_C", "02_A", "03_D", "04_M"])
        x_var = st.selectbox("Seleccione Variable Independiente (X)", data.columns)

        if st.button("Calcular Regresión Lineal"):
            fig, r2, R = plot_linear_regression(data_cuant, x_var, y_var)
            if fig:
                st.plotly_chart(fig, use_container_width=True)
                st.metric("Coeficiente de Determinación (R²)", f"{r2:.4f}")
                st.metric("Coeficiente de Correlación (R)", f"{R:.4f}")

    elif selected == "Regresión Lineal Múltiple":
        st.subheader("📊 Análisis de Regresión Lineal Múltiple")

        # Select Box para seleccionar la variable dependiente
        y_var = st.selectbox("Seleccione la Variable Dependiente (Y)", data.columns)

        # CheckBox para mostrar mapa de calor de las correlaciones más altas (>0.2)
        if st.checkbox("Mostrar Mapa de Calor de Correlaciones Altas"):
            corr_data = abs(data_cuant.corr()[y_var])
            corr_data = corr_data[corr_data > 0.1].sort_values(ascending=False).to_frame()

            if not corr_data.empty:
                st.dataframe(corr_data)
                fig, ax = plt.subplots(figsize=(8, 6))
                sns.heatmap(corr_data, cmap='Greens', annot=True, fmt=".2f", ax=ax)
                st.pyplot(fig)
            else:
                st.warning("No hay correlaciones superiores a 0.2 para la variable dependiente seleccionada.")

        # Multiselect para seleccionar las variables independientes
        x_vars = st.multiselect("Seleccione las Variables Independientes (X)", data.columns, default=None)

        # Realizar la regresión lineal múltiple y mostrar los coeficientes
        if x_vars and st.button("Calcular Regresión Lineal Múltiple"):
            X = data_cuant[x_vars]
            y = data_cuant[y_var]

            from sklearn.linear_model import LinearRegression
            model = LinearRegression()
            model.fit(X, y)
            y_pred = model.predict(X)

            # Calcular el coeficiente de determinación (R^2) y el coeficiente de correlación
            r2 = r2_score(y, y_pred)
            R = np.sqrt(r2)

            st.metric("Coeficiente de Determinación (R²)", f"{r2:.4f}")
            st.metric("Coeficiente de Correlación (R)", f"{R:.4f}")

            # Mostrar scatterplot de comparación entre valores reales y predichos
            fig = px.scatter(x=y, y=y_pred, labels={'x': 'Valores Reales', 'y': 'Valores Predichos'},
                             title=f"Comparación entre {y_var} Real y Predicho")
            fig.add_shape(type="line", x0=y.min(), x1=y.max(), y0=y.min(), y1=y.max(), line=dict(color='red', dash='dash'))
            st.plotly_chart(fig, use_container_width=True)

    elif selected == "Visualizaciones Adicionales":
        st.subheader("🎨 Visualizaciones Adicionales")

        # Visualización de boxplot
        col1, col2 = st.columns(2)
        with col1:
            y_box_var = st.selectbox("Seleccione Variable para Boxplot", data.columns)
            fig_box = plot_boxplot(data_cuant, y_box_var)
            st.plotly_chart(fig_box, use_container_width=True)

        # Visualización de scatterplot matrix
        with col2:
            selected_vars = st.multiselect("Seleccione Variables para Matriz de Dispersión", data.columns)
            if selected_vars:
                st.subheader("📊 Matriz de Dispersión")
                pairplot_fig = sns.pairplot(data_cuant[selected_vars])
                st.pyplot(pairplot_fig)

        # Visualización de histogramas con curvas de densidad
        st.subheader("📈 Histogramas con Curvas de Densidad")
        selected_hist_vars = st.multiselect("Seleccione Variables para Histogramas", data.columns)
        for hist_var in selected_hist_vars:
            fig_hist = px.histogram(data_cuant, x=hist_var, histnorm='probability density', title=f"Histograma de {hist_var}")
            st.plotly_chart(fig_hist, use_container_width=True)

    elif selected == "Análisis Univariado":
        st.subheader("📊 Análisis Univariado")


        # Cargar el nuevo DataFrame de SocioLimpio
        socio_data = load_socio_data()

        # Obtener las columnas categóricas disponibles del nuevo DataFrame
        cat_vars = socio_data.select_dtypes(include=['object']).columns

        # Seleccionar variable categórica
        cat_var = st.selectbox("Seleccione una variable categórica para analizar", cat_vars)

        # Calcular frecuencias y porcentajes
        freq = socio_data[cat_var].value_counts()
        percent = socio_data[cat_var].value_counts(normalize=True) * 100

        # Mostrar opciones de Frecuencia y Porcentaje
        option = st.radio("Seleccione lo que desea visualizar:", ["Frecuencia", "Porcentaje"])

        if option == "Frecuencia":
            st.write(f"Frecuencia de {cat_var}:")
            st.dataframe(freq)
        elif option == "Porcentaje":
            st.write(f"Porcentaje de {cat_var}:")
            st.dataframe(percent)

        # Gráficos: Barras y Pastel
        if st.checkbox("Mostrar Gráfico de Barras"):
            fig = px.bar(freq, x=freq.index, y=freq.values, labels={'x': cat_var, 'y': 'Frecuencia'}, title=f"Frecuencia de {cat_var}", color_discrete_sequence=['#00CC96'])
            st.plotly_chart(fig)

        if st.checkbox("Mostrar Gráfico de Pastel"):
            fig = go.Figure(data=[go.Pie(labels=freq.index, values=freq.values)])
            fig.update_traces(hoverinfo='label+percent', textinfo='value+percent')
            st.plotly_chart(fig)

    elif selected == "Regresión logística":
        st.subheader("📊 Regresión logística")

        # Select box para seleccionar la variable dependiente dicotómica (Y)
        y_variable = st.selectbox("Selecciona la variable dependiente (Y)", data.columns)

        # Multiselect para seleccionar las variables numéricas independientes (X)
        x_variables = st.multiselect("Selecciona las variables independientes (X)", data.columns)

        if y_variable and x_variables:
            # Separar los datos en X e Y usando data_cuant
            X = data_cuant[x_variables]  # Variables independientes
            y = data_cuant[y_variable]   # Variable dependiente

            # Dividir los datos en conjunto de entrenamiento y prueba
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

            # Crear el modelo de regresión logística
            logistic_model = LogisticRegression()
            logistic_model.fit(X_train, y_train)

            # Predecir con el modelo
            y_pred = logistic_model.predict(X_test)


            # Calcular la matriz de confusión
            cm = confusion_matrix(y_test, y_pred)
            st.write("Matriz de Confusión:")
            st.write(cm)

            # Extraer los valores de la matriz de confusión
            TN, FP, FN, TP = cm.ravel()  # Descomponer la matriz de confusión

            # Mostrar la matriz de confusión con sus componentes
            st.write("Detalles de la Matriz de Confusión:")
            st.write(f"Verdaderos Positivos (TP): {TP}")
            st.write(f"Falsos Negativos (FN): {FN}")
            st.write(f"Falsos Positivos (FP): {FP}")
            st.write(f"Verdaderos Negativos (TN): {TN}")

            # Calcular precisión, exactitud y sensibilidad
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)

            # Mostrar los resultados
            st.write(f"Precisión: {precision:.2f}")
            st.write(f"Exactitud: {accuracy:.2f}")
            st.write(f"Sensibilidad: {recall:.2f}")




if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.69.255.241



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.69.255.241:8501

your url is: https://legal-taxes-sin.loca.lt
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1119: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1119: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1119: FutureWarning:

use_inf_as_na option is

In [8]:
%%writefile app.py

Overwriting app.py
